In [51]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [52]:
from langchain.tools import tool
from typing import List, Dict
from langchain_teddynote.tools import GoogleNews

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_teddynote.messages import AgentStreamParser

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_community.agent_toolkits import FileManagementToolkit
import os

In [53]:
@tool
def latest_news(k: int = 5) -> List[Dict[str, str]]:        # 최신 뉴스를 검색하는 도구
    """Look up latest news"""

    print(f'최신 뉴스 검색')

    news_tool = GoogleNews()

    return news_tool.search_latest(k=k)                     # 최신 뉴스 k개를 검색해서 리턴

In [54]:
if not os.path.exists('tmp3'):                              # 작업 폴더 생성
    os.mkdir('tmp3')

working_directory = 'tmp3'                                   # 작업 디렉토리 설정 

tools = FileManagementToolkit(                              # 파일 관리 도구를 가져온다
    root_dir=str(working_directory)
).get_tools()


tools.append(latest_news)                                   # latest_news 도구를 tools에 추가한다

In [55]:
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

In [56]:
prompt = ChatPromptTemplate.from_messages(                  # 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. 도구의 역할과 이름을 입력을 입력합니다.
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `latest_news` tool to find latest news. "         
            "Make sure to use the `file_management` tool to manage files. ",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [57]:
agent = create_tool_calling_agent(llm, tools, prompt)       # agent 생성

In [58]:
agent_executor = AgentExecutor(                             # AgentExecutor 생성 
    agent=agent,                                            # 각 단계에서 계획을 생성하고 행동을 결정하는 agent
    tools=tools,                                            # agent 가 사용할 수 있는 도구 목록
    verbose=True,
    max_iterations=10,                                      # 최대 10번 까지 반복
    max_execution_time=10,                                  # 실행되는데 소요되는 최대 시간
    handle_parsing_errors=True      
)

In [59]:
store = {}                              # 채팅 기록을 위한 메모리

def get_session_history(session_ids):   # session_id 를 기반으로 세션 기록을 가져오는 함수    
    if session_ids not in store:        # session_id 가 store에 없는 경우        
        store[session_ids] = ChatMessageHistory()   # 새로운 ChatMessageHistory 객체를 딕셔너리에 저장
    
    return store[session_ids]           # 해당 세션 ID에 대한 세션 기록 리턴


agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,                                    # session_id 를 기반으로 세션 기록을 가져오는 함수
    input_messages_key='input',                             # 프롬프트의 질문으로 들어가는 key: input
    history_messages_key='chat_history'                     # 프롬프트의 대화 기록이 들어가는 key: chat_history
)

In [60]:
def tool_callback(tool) -> None:
    print(f'===== 도구 호출 =====')
    print(f"사용된 도구 이름 : {tool[0]}")
    print(f'===== 도구 호출 =====')

In [61]:
def obervation_callback(observation) -> None:
    print(f'\n===== 관찰 내용 =====')
    print(f"Observation: {observation[0].observation}")
    print(f'===== 관찰 내용 =====')

In [62]:
def result_callback(result: str) -> None:
    print(f'===== 최종 답변 =====')
    print(f"{result}")
    print(f'===== 최종 답변 =====')

In [63]:
result = agent_with_chat_history.stream(
    {
        'input': '최신 뉴스 5개를 검색하고, 각 뉴스의 제목을 파일명으로 하는 파일명을 생성하고(.txt), '
        '파일의 뉴스 내용은 뉴스의 내용과 url을 추가하세요.'
    }, 
    config={'configurable': {'session_id': 'abc123'}}            
)

In [64]:
for step in result:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)  



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : tool='latest_news' tool_input={'k': 5} log="\nInvoking: `latest_news` with `{'k': 5}`\n\n\n" message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_MUZULUbH0mT93qKN4sMC2RgR', 'function': {'arguments': '{"k":5}', 'name': 'latest_news'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1'}, id='run-3b4b95fe-5ff8-42bf-8e4b-93bee1001abb', tool_calls=[{'name': 'latest_news', 'args': {'k': 5}, 'id': 'call_MUZULUbH0mT93qKN4sMC2RgR', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'latest_news', 'args': '{"k":5}', 'id': 'call_MUZULUbH0mT93qKN4sMC2RgR', 'index': 0, 'type': 'tool_call_chunk'}])] tool_call_id='call_MUZULUbH0mT93qKN4sMC2RgR'
===== 도구 호출 =====

Invoking: `latest_news` with `{'k': 5}`


최신 뉴스 검색
[{'url': 'https://news.google.com/rss/articles/CBMihgFBVV95cUxQWkp2Z3phZmh2Ty1Uc2ZXQ2J

Stopping agent prematurely due to triggering stop condition


===== 도구 호출 =====
사용된 도구 이름 : tool='write_file' tool_input={'file_path': '하늘이 살해 교사.txt', 'text': '하늘이 살해 교사, ‘흉기’ ‘살인사건 기사’ 검색…경찰 “계획범행 무게” - 조선일보\nURL: https://news.google.com/rss/articles/CBMihgFBVV95cUxQWkp2Z3phZmh2Ty1Uc2ZXQ2JzT1hJUzJxdC1GWmM4RkVUb2g3RVdPbFJualM3Z04zd05jbVhfenMxb056ZFZ3ZHJ6MnMtNDhSaFVtS3NQUEFOcUt2Mk1MZ3M3RmNmZGhZXzNXMFJJOGFZdHY0czlSbjZYRjZuakR6SzY1SHR1QdIBmgFBVV95cUxQTlNyaXhWSExqMjJoTnl3dkliV3RDYVpDTnphdnRRYnF0dW1Gc0h4LTY0emZ5dFBRLVhTeU43ckVsVDlvYllpUUYxUFNMRFFjdHl3Vk1RWFpwQ0kxZlpmbFF3QUdoa0xCUDBLcVoyTmxrc21iQXhwZ2FXbzFGVmIzVWRLcHpDVFlCNzJ3VkQ3S3hkaUNHS0Mxekh3?oc=5'} log="\nInvoking: `write_file` with `{'file_path': '하늘이 살해 교사.txt', 'text': '하늘이 살해 교사, ‘흉기’ ‘살인사건 기사’ 검색…경찰 “계획범행 무게” - 조선일보\\nURL: https://news.google.com/rss/articles/CBMihgFBVV95cUxQWkp2Z3phZmh2Ty1Uc2ZXQ2JzT1hJUzJxdC1GWmM4RkVUb2g3RVdPbFJualM3Z04zd05jbVhfenMxb056ZFZ3ZHJ6MnMtNDhSaFVtS3NQUEFOcUt2Mk1MZ3M3RmNmZGhZXzNXMFJJOGFZdHY0czlSbjZYRjZuakR6SzY1SHR1QdIBmgFBVV95cUxQTlNyaXhWSExqMjJoTnl3dkliV3RDYVpDTnph

In [65]:
result = agent_with_chat_history.stream(
    {
        'input': '이전에 생성한 파일 제목 맨 앞에 제목에 어울리는 emoji를 추가하여 파일명을 변경하세요. '
        '파일도 깔끔하게 변경하세요.'
    }, 
    config={'configurable': {'session_id': 'abc123'}}            
)

In [66]:
for step in result:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)  



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : tool='list_directory' tool_input={} log='\nInvoking: `list_directory` with `{}`\n\n\n' message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_2CfIZycomSacFKUEby96M6jZ', 'function': {'arguments': '{}', 'name': 'list_directory'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1'}, id='run-f754c79d-bada-42f0-8c52-b4f5ce374b4b', tool_calls=[{'name': 'list_directory', 'args': {}, 'id': 'call_2CfIZycomSacFKUEby96M6jZ', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'list_directory', 'args': '{}', 'id': 'call_2CfIZycomSacFKUEby96M6jZ', 'index': 0, 'type': 'tool_call_chunk'}])] tool_call_id='call_2CfIZycomSacFKUEby96M6jZ'
===== 도구 호출 =====

Invoking: `list_directory` with `{}`


내가 김용현한테 ‘챙기라.txt
내일 윤석열 첫 형사재판.txt
트럼프 “협상 못끼어 배신감.txt
하늘이 살해 교사.txt
헌재, ‘한덕수 총리 탄핵 정족수.txt
===== 관찰 내용 =====


In [67]:
result = agent_with_chat_history.stream(
    {
        'input': '이전에 생성한 모든 파일을 news 폴더를 생성한 뒤 해당 폴더에 모든 파일을 복사하세요. '
        '내용도 동일하게 복사하세요.'
    }, 
    config={'configurable': {'session_id': 'abc123'}}            
)

In [68]:
for step in result:
    if 'actions' in step:
        tool_callback(step['actions'])
    elif 'steps' in step:
        obervation_callback(step['steps'])
    elif 'output' in step:
        result_callback(step['messages'][0].content)  



> Entering new None chain...
===== 도구 호출 =====
사용된 도구 이름 : tool='list_directory' tool_input={} log='\nInvoking: `list_directory` with `{}`\n\n\n' message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_l04tgkey9SxCKz5nIok2vBTm', 'function': {'arguments': '{}', 'name': 'list_directory'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1'}, id='run-e025f54b-70fc-438b-9676-346be21fa97b', tool_calls=[{'name': 'list_directory', 'args': {}, 'id': 'call_l04tgkey9SxCKz5nIok2vBTm', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'list_directory', 'args': '{}', 'id': 'call_l04tgkey9SxCKz5nIok2vBTm', 'index': 0, 'type': 'tool_call_chunk'}])] tool_call_id='call_l04tgkey9SxCKz5nIok2vBTm'
===== 도구 호출 =====

Invoking: `list_directory` with `{}`


☁️ 하늘이 살해 교사.txt
⚖️ 내일 윤석열 첫 형사재판.txt
⚖️ 헌재, ‘한덕수 총리 탄핵 정족수.txt
🇺🇸 트럼프 “협상 못끼어 배신감.txt
😀 내가 김용현한테 ‘챙기라.txt
====

Stopping agent prematurely due to triggering stop condition


===== 도구 호출 =====
사용된 도구 이름 : tool='copy_file' tool_input={'source_path': '☁️ 하늘이 살해 교사.txt', 'destination_path': 'news/☁️ 하늘이 살해 교사.txt'} log="\nInvoking: `copy_file` with `{'source_path': '☁️ 하늘이 살해 교사.txt', 'destination_path': 'news/☁️ 하늘이 살해 교사.txt'}`\n\n\n" message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_G4j2XnbSQsqUu2LvkjlEZ5mU', 'function': {'arguments': '{"source_path": "☁️ 하늘이 살해 교사.txt", "destination_path": "news/☁️ 하늘이 살해 교사.txt"}', 'name': 'copy_file'}, 'type': 'function'}, {'index': 1, 'id': 'call_Q6KLTqDHbHULQAau7FxbRvgA', 'function': {'arguments': '{"source_path": "⚖️ 내일 윤석열 첫 형사재판.txt", "destination_path": "news/⚖️ 내일 윤석열 첫 형사재판.txt"}', 'name': 'copy_file'}, 'type': 'function'}, {'index': 2, 'id': 'call_NBMuvuKxnLIVPMa942U6xBWz', 'function': {'arguments': '{"source_path": "⚖️ 헌재, ‘한덕수 총리 탄핵 정족수.txt", "destination_path": "news/⚖️ 헌재, ‘한덕수 총리 탄핵 정족수.txt"}', 'name': 'copy_file'}, 'type': 'function'}, {'index': 3, 'id': 'ca

In [70]:
from langchain_teddynote.messages import AgentStreamParser

In [71]:
agent_stream_parser = AgentStreamParser()

result = agent_with_chat_history.stream(
    {"input": "news 폴더를 제외한 모든 .txt 파일을 삭제하세요."},
    config={"configurable": {"session_id": "abc123"}},
)

print("Agent 실행 결과:")

for step in result:
    agent_stream_parser.process_agent_steps(step)

Agent 실행 결과:


> Entering new None chain...
[도구 호출]
Tool: file_search
pattern: *.txt
Log: 
Invoking: `file_search` with `{'pattern': '*.txt'}`




Invoking: `file_search` with `{'pattern': '*.txt'}`


☁️ 하늘이 살해 교사.txt
⚖️ 내일 윤석열 첫 형사재판.txt
⚖️ 헌재, ‘한덕수 총리 탄핵 정족수.txt
🇺🇸 트럼프 “협상 못끼어 배신감.txt
😀 내가 김용현한테 ‘챙기라.txt
news\README.txt
news\☁️ 하늘이 살해 교사.txt
news\⚖️ 내일 윤석열 첫 형사재판.txt
news\⚖️ 헌재, ‘한덕수 총리 탄핵 정족수.txt
news\🇺🇸 트럼프 “협상 못끼어 배신감.txt
news\😀 내가 김용현한테 ‘챙기라.txt[관찰 내용]
Observation: ☁️ 하늘이 살해 교사.txt
⚖️ 내일 윤석열 첫 형사재판.txt
⚖️ 헌재, ‘한덕수 총리 탄핵 정족수.txt
🇺🇸 트럼프 “협상 못끼어 배신감.txt
😀 내가 김용현한테 ‘챙기라.txt
news\README.txt
news\☁️ 하늘이 살해 교사.txt
news\⚖️ 내일 윤석열 첫 형사재판.txt
news\⚖️ 헌재, ‘한덕수 총리 탄핵 정족수.txt
news\🇺🇸 트럼프 “협상 못끼어 배신감.txt
news\😀 내가 김용현한테 ‘챙기라.txt
[도구 호출]
Tool: file_delete
file_path: ☁️ 하늘이 살해 교사.txt
Log: 
Invoking: `file_delete` with `{'file_path': '☁️ 하늘이 살해 교사.txt'}`



[도구 호출]
Tool: file_delete
file_path: ⚖️ 내일 윤석열 첫 형사재판.txt
Log: 
Invoking: `file_delete` with `{'file_path': '⚖️ 내일 윤석열 첫 형사재판.txt'}`



[도구 호